# GIS Visualizations with Folium and GeoPandas 

This is a demonstration of how open source visualization tools can be applied to the civic participation space. The result is a self-contained map of Washington County, Oregon, that contains easy-to-access information about a user's state house district and 'home' HDLs.

## Set up

Install any packages that you don't have and import them for use.
You will need `geopandas` and `folium` (plus dependencies)

In [ ]:
!pip install geopandas folium

#install them via command line with `!`

In [ ]:
import geopandas as gpd
import folium

You will need to get the Oregon districts GIS file. 
* The 2021 redistricting shapefiles are here: https://geo.maps.arcgis.com/home/group.html?id=cffa41c3de414f6587459df048d728b5#overview

In [ ]:
# Read in data. There are multiple files (Federal, State House, State Senate). 
# You will need the one for the State House

shp_file='House_SB_882.shp' #this file is in the same directory as the notebook
gdf = gpd.read_file(shp_file) 

In [ ]:
# This file needs to be reprojected to Lat/Long: http://epsg.io/4326
# This is ESSENTIAL, as without this conversion, the latlong will be completely wrong!
gdf_4326 = gdf.to_crs(epsg='4326')

In [ ]:
#specify the new geojson name
json_file='oregon_lower.json'

# Write converted file to disk for backup
gdf_4326.to_file(json_file, driver="GeoJSON")

In [ ]:
# Read GeoJSON file into GeoPandas dataframe
# We're doing it from disk for redundancy, though technically you can continue in the workflow by 
# calling gdf_4326

gdf = gpd.read_file("oregon_lower.geojson")

Some data exploration below for the Oregon districts data. In order to make the map, you need to identify the column name for the district number, so that we can merge the HDL information on this field for later.

In this case, the relevant (or at least fastest) identifier for `gdf` is `DISTRICT`.

In [ ]:
gdf.info()

Now we're going to import our HDL information. I made a CSV for it (`hdl_names_anon.csv`), and it lives in the same directory as the notebook. 

This CSV has two fields: `DISTRICT` for the House district number, and `HDL` for HDL names. You can edit this in a text editor or in a spreadsheet editor. Note that if you're editing it bare in a text editor, you will need to wrap multiple names (and commas) in double quotes. 

In [ ]:
import pandas as pd 

#import HDL information, which was collected and then entered by hand
#This version for Github has been somewhat anonymized

hdl = pd.read_csv("hdl_names_anon.csv")
hdl.info()

In [ ]:
hdl.head(20)

Merge the info in gdf (`oregon_lower.geojson`) and hdl (`hdl_names_anon.csv`) together on the column `DISTRICT` to create a unified dataframe.

In [ ]:
everything = gdf.merge(hdl, on="DISTRICT")
everything.info()

## Visualization procedure

In [ ]:
# Initiates a map that's centered on the lat/long for the main office of the WashCoDems at the specified zoom level
m = folium.Map(location=[45.5231839,-122.98797], zoom_start=9.5)

In [ ]:
# See how the map looks at this stage, optional
m

Now read in GIS information for the WashCo boundaries. The original map has the boundaries, but they can't been see very clearly.

In [ ]:
washco = gpd.read_file('washco.json')

Apply it to the map `m` that we just created with black boundaries. It is now also lightly filled in white.

In [ ]:
folium.GeoJson(washco, name='County Boundaries',control=False,
                       style_function=lambda x: {"weight":3, 'color':'black','fillColor':'white'}).add_to(m)

In [ ]:
# see how the map looks after adding boundaries, again optional
m

This next step creates almost the entire visualization.

It reads in the information in `everything` (the dataframe), and
1. applies house district boundary lines in dark blue
2. colors in districts that have an HDL (if the HDL field isn't marked as 'Vacant!') in blue, white if there is no HDL
3. Add a tooltip when you place your mouse over it with the House District # and the HDL names
4. adds it to the existing map `m`

In [ ]:
folium.features.GeoJson(everything, name="House Districts", 
                        style_function=lambda x: {"weight":2,'color':'blue',
                                                  'opacity':0.5,'fillColor':'blue' 
                                                      if x['properties']['HDL'] != 'Vacant!' 
                                                      else 'white',
                                                  'fillOpacity':'0.1'},
                        tooltip=folium.features.GeoJsonTooltip(['DISTRICT','HDL'], 
                                                       aliases=['House District #', 'HDL(s)'])).add_to(m)

In [ ]:
# Now see for yourself!
m

In [ ]:
#Add controls to map

folium.LayerControl(autoZIndex=False, collapsed=False).add_to(m)

In [ ]:
#add full screen option to map
from folium import plugins

plugins.Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)

In [ ]:
# optional: see what it looks like now
m

In [ ]:
# saves the map to a self-contained webpage in the same directory as the notebook.
m.save("prototype.html")   

And you're done. Bye!